In [47]:
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
import json

#### load trained model from HLTM

In [48]:
vobs = []
with open('/Users/zhengfang/Library/CloudStorage/OneDrive-Personal/Projects/HITLTM_reddit/QDTM/input/vobs.txt', 'r') as file:
    for line in file.readlines():
        vobs.append(line.strip())

topic_word = []
with open('/Users/zhengfang/Library/CloudStorage/OneDrive-Personal/Projects/HITLTM_reddit/QDTM/temp/model4/nzw.txt', 'r') as file:
    for line in file.readlines():
        words = line.strip().split()
        if len(words) < len(vobs):
            topic_word.append(words + [0]*(len(vobs)-len(words)))
        else:
            topic_word.append(words)
topic_word = np.array(topic_word).astype(float)

In [50]:
# print your topics
topic_words = []
for i, topic in enumerate(topic_word):
    top_words = [re.sub("\$\d*", "", vobs[index]) for index in topic.argsort()[:-10 - 1:-1]]
    topic_words.append(top_words)
    print ("topic " + str(i+1) + ":\t" + " ".join(topic_words[i]))

topic 1:	video tutor apply find work check email link company interview
topic 2:	native english speaker language hire teach teacher south country accent
topic 3:	pay tax company work make income money year state minimum
topic 4:	class pay hour minute base bonus teach rate month min
topic 5:	cancellation soft absence note doctor penalty free cancel submit appointment
topic 6:	hour week work rating day time ph schedule weekend make
topic 7:	fuck de make grammar swear spam ban offense insult worthy
topic 8:	month contract pay year start change sign end day wait
topic 9:	student give lesson rating level class time bad good make
topic 10:	class feedback parent send time leave write happen ticket teacher
topic 11:	account bank paypal payoneer pay payment transfer fee receive transferwise
topic 12:	teach online tefl degree experience english company certificate year work
topic 13:	culture en learn par norm behavior question serve est leave
topic 14:	talent field feature feb february federal f

#### covert documents into doc topic distributions

In [51]:
def load_data(path, processor):
    titles = []
    authors = []
    original_texts = []
    processed_texts = []
    with open(path, "r") as file:
        for line in file.readlines():
            sample = json.loads(line)
            titles.append(sample['title'])
            # titles.append(sample['text'][:20])
            authors.append(sample['author'])
            # authors.append("none")
            original_texts.append(sample['text'])
            processed_texts.append(processor.preprocess(sample['text']))
    return titles, authors, original_texts, processed_texts

In [52]:
from preprocessor import read_stopwords, preprocessor
stopwords = read_stopwords("./dataset/stopwords.en.txt")
processor = preprocessor(stopwords)

text_path = "/Users/zhengfang/Library/CloudStorage/OneDrive-Personal/Projects/HITLTM_reddit/dataset/reddit_texts.json"
titles, authors, original_texts, processed_texts = load_data(text_path, processor)

In [53]:
vectorizer = CountVectorizer(vocabulary=vobs)
bow_vectors = vectorizer.transform(processed_texts).toarray()

pzw = topic_word/topic_word.sum(0)
results = (bow_vectors @ pzw.T)
temp = results + 1e-20
x = temp / temp.sum(1).reshape((temp.shape[0], 1))

In [55]:
x.shape

(9651, 14)

In [46]:
for idx in range(x.shape[1]):
    rnks = np.argsort(x[idx])[::-1]
    print(f"topic {idx}")
    print(f"topic words: {topic_words[idx]}")
    print("Ranked documents (from most relevant to least):")
    print(np.array(original_texts)[rnks])

topic 0
topic words: ['video', 'tutor', 'apply', 'find', 'work', 'check', 'email', 'link', 'company', 'interview']
Ranked documents (from most relevant to least):
['You will need to pay 15% for self employment taxes (social security and FICA even before income taxes).  This is a dude gig for me. I paid a total (including the self employment taxes) of about 32% to the federal government.My state has a flat 5% income tax.\n\nIn total, I paid about 37% of my income in taxes.'
 "You are self employed and you must declare your income to the HMRC and therefore the DWP, who will then tell you what you need to do, however, then you are at risk of losing your benefits. But regardless, you will be committing benefit fraud and evading tax if you don't pay or declare.\n\nWhile I wouldn't recommend it, it is highly unlikely you'd get caught over such small amounts. DWP don't have the manpower to check accounts and wouldn't bother if you're claiming the bare minimum benefits like job seekers. I'd be

In [41]:
rnks

array([ 2,  3,  7, 11,  8,  5,  1,  0,  9, 10, 13, 12,  6,  4])